In [7]:
search_for = 80
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.002472400665283203
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 350
all cases: 156814647
type: [1, 1, 1, 1, 80] 80
cases of this type: 40960000
10000 26.22495372628959
20000 31.092274248728877
30000 100.11101161201323
40000 82.00003899547126
50000 84.06756302132308
60000 85.55227480267006
70000 83.19401568535211
80000 81.76002600223492
90000 77.77486145006345
100000 79.8001345914518
110000 84.45175590494114
120000 79.58606210459168
130000 87.58345240504522
140000 91.7436977042163
150000 83.16549735248918
160000 81.99591120158827
170000 78.38396678869674
180000 77.57193027163412
190000 77.90877332687556
200000 83.11228484844459
210000 80.39331445907628
220000 82.97778206445751
230000 74.70382765238034
240000 85.8650477096765
250000 40.063786633350674
260000 13.934948864891217
270000 13.945966922723947
280000 54.85494616449906
290000 87.90051971246793
300000 73.9790804085283
310000 79.44671575026348
320000 

3110000 24.337570970445864
3120000 26.33044087776369
3130000 29.04613526378434
3140000 71.26434361454605
3150000 62.798812357592126
3160000 65.99357649500298
3170000 67.29030558113575
3180000 64.09178523636699
3190000 63.570110057803966
3200000 64.03347267315493
3210000 64.65936987631274
3220000 64.53951502562226
3230000 70.5803335974274
3240000 62.30819015897702
3250000 66.7045745867727
3260000 65.03587561370432
3270000 60.54483002189505
3280000 65.74589687979869
3290000 65.1473415879526
3300000 58.691031553800705
3310000 61.37424472754071
3320000 42.90772320126476
3330000 13.092017042960743
3340000 13.018491887037602
3350000 33.85942900459575
3360000 66.2210824365033
3370000 55.82868105236674
3380000 61.714044354543674
3390000 61.66560984910935
3400000 64.99646524150296
3410000 58.893208270496366
3420000 64.09401964935395
3430000 62.769915081751904
3440000 68.03050050942338
3450000 66.14842064479625
3460000 63.255242878081
3470000 65.20662640098206
3480000 61.031625481673956
3490000 

6240000 24.49347668583999
6250000 75.53974481852823
6260000 67.14207403354418
6270000 59.32192703878748
6280000 66.03465666757278
6290000 59.702230227269474
6300000 63.199206333328746
6310000 67.2247229083832
6320000 64.98438211738541
6330000 60.97682514259917
6340000 58.89500442496911
6350000 56.64795318041339
6360000 56.2373632600314
6370000 55.82240459689182
6380000 54.93552149722384
6390000 53.77247127661037
6400000 12.862425992102775
6410000 13.070915241593156
6420000 18.584563115892948
6430000 60.9170828535653
6440000 55.13780298294497
6450000 58.103206568874654
6460000 57.962063030426066
6470000 55.02666801759477
6480000 58.315961733196254
6490000 59.74863905730861
6500000 65.1301546589039
6510000 62.434395522636464
6520000 59.33000022228464
6530000 63.826257126950786
6540000 63.16692605380619
6550000 64.48387273714967
6560000 62.10490827715206
6570000 59.63577576649712
6580000 30.373811338733795
6590000 29.588315946645192
6600000 36.01867232426756
6610000 71.8739206395527
66200

9370000 52.99715856208432
9380000 48.067239580501145
9390000 57.956641079264436
9400000 48.39590436980642
9410000 54.05687854014911
9420000 52.48368687623012
9430000 52.1545796214065
9440000 56.864125780689555
9450000 47.57119562834498
9460000 57.753865765303324
9470000 12.2555018023739
9480000 11.506928504451116
9490000 12.215516985428161
9500000 61.95865713381317
9510000 53.907213800848425
9520000 56.70224427950939
9530000 49.33681473301335
9540000 54.88990406214744
9550000 54.496845994058276
9560000 51.023086564598486
9570000 54.50495669583005
9580000 52.570773538671524
9590000 58.67441844285818
9600000 62.606179783061506
9610000 27.3709016826197
9620000 30.430810052774124
9630000 28.686688732822223
9640000 56.57002725089826
9650000 56.27240575545714
9660000 63.47662458892812
9670000 63.99150828405769
9680000 60.504106489570496
9690000 55.38079815891964
9700000 58.70898634876983
9710000 62.709204236851164
9720000 61.128054380705976
9730000 66.7543072949192
9740000 70.5652583469705
9

12410000 55.44620818478506
12420000 51.46221103063613
12430000 56.82745531195513
12440000 16.681390136982365
12450000 21.81434617617581
12460000 23.025958579063747
12470000 52.28004643771395
12480000 51.048045866535126
12490000 56.6264641337182
12500000 61.09522107427404
12510000 49.858607697292804
12520000 49.8964824589014
12530000 54.785627487120465
12540000 21.3374079141378
12550000 11.560605603871027
12560000 12.250013248255286
12570000 53.425542177977626
12580000 52.48560857744715
12590000 54.3759997523121
12600000 52.275462675934826
12610000 65.29408719219467
12620000 49.93675291147933
12630000 57.30172524503039
12640000 35.68167387593275
12650000 28.561369677744054
12660000 29.79825684575504
12670000 44.40596036845746
12680000 50.78496035657438
12690000 43.47127309120772
12700000 58.80481166897128
12710000 54.62077110965641
12720000 53.96224905830218
12730000 53.35822644835076
12740000 58.50493300673264
12750000 63.3415702369343
12760000 65.19788472849092
12770000 54.67917271646

...

147940000 1.8305141274435388
147950000 0.5905954149817957
147960000 1.7188726063354016
147970000 1.3455619313700131
147980000 2.0104495736983683
147990000 1.841382927527944
148000000 1.6705630619074967
148010000 1.3187947977151673
148020000 1.2346672015914917
148030000 1.0224295579105285
148040000 1.043279619467861
148050000 1.7963798916848104
148060000 1.5411155895956026
148070000 1.9369211200985774
148080000 1.0678946534030636
148090000 1.793959524854905
148100000 1.0736599721558557
148110000 1.3913703280623555
148120000 1.4176107366370096
148130000 0.8135683929884964
148140000 0.8713142236083545
148150000 1.362459450563974
148160000 1.2167980801599092
148170000 1.3400749022649328
148180000 1.4932505412173338
148190000 1.026692536528713
148200000 1.458025737236202
148210000 1.390990497954064
148220000 0.8682416994132663
148230000 1.7235346371974944
148240000 1.643622473947631
148250000 1.262827223290152
148260000 1.1311630445521077
148270000 0.7654716323096487
148280000 1.21225765114

150750000 0.4529243288126985
150760000 0.455236403688497
150770000 0.45279352776063153
150780000 0.4539338332726161
150790000 0.452673378661063
150800000 0.5383489279898935
150810000 0.7408856302466392
150820000 0.766906299602727
150830000 0.5015587089445392
150840000 0.4986962680328289
150850000 0.5309881666910582
150860000 0.6708757546829516
150870000 0.8307147955570022
150880000 0.7844198388270471
150890000 0.31902879294782877
150900000 0.3844263837890625
150910000 0.6351721407285068
150920000 0.9125669216022956
150930000 0.537504117441376
150940000 0.8847534080382852
150950000 0.6161715532399681
150960000 0.3093119782001575
150970000 0.4908915784549647
150980000 0.6637979441926215
150990000 0.6860955679066181
151000000 0.4776169645530052
151010000 0.814825044359313
151020000 0.327837490006725
151030000 0.3688298731202947
151040000 0.5810330771181319
151050000 0.6397108973727823
151060000 0.55066151624112
151070000 0.5608826290472282
151080000 0.41437797158968454
151090000 0.2657534

153390000 0.3166195795403719
153400000 0.28085534569431675
153410000 0.20242244418759478
153420000 0.2620600560722351
153430000 0.139797483369569
153440000 0.2282050705030892
153450000 0.21838709656025967
153460000 0.14635486378407478
153470000 0.13838111900883915
153480000 0.13260597137941918
153490000 0.13213939300769567
153500000 0.13116007834016613
153510000 0.13286049353039264
153520000 0.13212906366799937
153530000 0.13080591895606122
153540000 0.13148954711512725
153550000 0.12977075553443035
153560000 0.1290552131220632
153570000 0.1286763291455706
153580000 0.2638366687437958
153590000 0.36894376831705705
153600000 0.32299499985229974
153610000 0.3400871049979727
153620000 0.274853760642959
153630000 0.22329233644209306
153640000 0.2558662147690654
153650000 0.2599585094423029
153660000 0.19899264515356224
153670000 0.21709764161403314
153680000 0.38040206758421
153690000 0.2741365503414273
153700000 0.37041065862672196
153710000 0.49637849504673476
153720000 0.480794369748969

155610000 0.10776224216280379
155620000 0.0620522090446088
155630000 0.053657322812643314
155640000 0.0566121485251983
best so far: 0
type: [2, 2, 1, 5, 4] 80
cases of this type: 64000
155650000 0.05564254187447494
155660000 0.061191817304220464
155670000 0.09946756044900418
155680000 0.05222970052460829
155690000 0.05162991544124153
155700000 0.050961462709069254
155710000 0.04932381819739606
best so far: 0
type: [2, 2, 1, 10, 2] 80
cases of this type: 32000
155720000 0.06613832876024643
155730000 0.06786936078228553
155740000 0.06803321909977993
best so far: 0
type: [2, 2, 1, 20, 1] 80
cases of this type: 16000
155750000 0.04846503107780881
155760000 0.03814595948660374
best so far: 0
type: [2, 2, 2, 1, 10] 80
cases of this type: 80000
155770000 0.06345437778803374
155780000 0.06379222786163621
155790000 0.048385831465800604
155800000 0.04103572799148825
155810000 0.07183562387802866
155820000 0.07132025764971972
155830000 0.050086111877606976
155840000 0.05681874280446767
best so fa

best so far: 0
type: [4, 10, 1, 1, 2] 80
cases of this type: 160
best so far: 0
type: [4, 10, 1, 2, 1] 80
cases of this type: 80
best so far: 0
type: [4, 10, 2, 1, 1] 80
cases of this type: 40
best so far: 0
type: [4, 20, 1, 1, 1] 80
cases of this type: 20
best so far: 0
type: [5, 1, 1, 1, 16] 80
cases of this type: 65536
156590000 0.009572447363436222
156600000 0.014815073780854544
156610000 0.013557782376375465
156620000 0.010157721820327972
156630000 0.007331132459084192
156640000 0.010050924408766959
156650000 0.01596924488602082
best so far: 0
type: [5, 1, 1, 2, 8] 80
cases of this type: 32768
156660000 0.009978369525551796
156670000 0.00722410369537274
156680000 0.005542704634937975
best so far: 0
type: [5, 1, 1, 4, 4] 80
cases of this type: 16384
156690000 0.005931401451746623
156700000 0.005294916656037172
best so far: 0
type: [5, 1, 1, 8, 2] 80
cases of this type: 8192
156710000 0.007422992625415325
best so far: 0
type: [5, 1, 1, 16, 1] 80
cases of this type: 4096
best so far: